# 예제 샘플 호텔 및 항공권 예약 에이전트

이 솔루션은 항공권과 호텔 예약을 도와줍니다. 시나리오는 2024년 2월 20일 런던 히드로 공항(LHR)에서 뉴욕 JFK로 출발하여 2025년 2월 27일에 돌아오는 일정이며, 브리티시 에어웨이즈의 이코노미 클래스를 이용하는 여행입니다. 뉴욕에서는 힐튼 호텔에 머물고 싶습니다. 항공권과 호텔 비용을 제공해 주세요.


# Azure AI Agent Service 초기화 및 **.env**에서 구성 정보 가져오기

### **.env**

.env 파일 생성

**.env** 파일에는 Azure AI Agent Service의 연결 문자열, AOAI에서 사용하는 모델, 그리고 해당 Google API Search 서비스 API, ENDPOINT 등이 포함됩니다.

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "Azure AI Agent Service 모델 배포 이름"

[**NOTE**] 분당 100,000 토큰의 Rate Limit과 분당 600 요청의 Rate Limit을 가진 모델이 필요합니다.

  모델은 Azure AI Foundry의 Model and Endpoint에서 얻을 수 있습니다.

- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "Azure AI Agent Service 프로젝트 연결 문자열"

  프로젝트 연결 문자열은 AI Foundry 포털 화면의 프로젝트 개요에서 확인할 수 있습니다.

- **SERPAPI_SEARCH_API_KEY** = "SERPAPI Search API KEY"
- **SERPAPI_SEARCH_ENDPOINT** = "SERPAPI Search Endpoint"

Azure AI Agent Service의 모델 배포 이름과 프로젝트 연결 문자열을 얻으려면 Azure AI Agent Service를 생성해야 합니다. [이 템플릿](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json)을 사용하여 직접 생성하는 것을 권장합니다. （***참고:*** Azure AI Agent Service는 현재 제한된 지역에서만 설정 가능합니다. [이 링크](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support)를 참조하여 지역을 설정하는 것을 권장합니다.)

Agent는 SERPAPI에 접근해야 합니다. [이 링크](https://serpapi.com/searches)를 통해 등록하는 것을 권장합니다. 등록 후 고유한 API KEY와 ENDPOINT를 얻을 수 있습니다.


# 설정

이 노트북을 실행하려면 `pip install -r requirements.txt` 명령어를 실행하여 필요한 라이브러리를 설치했는지 확인해야 합니다.


In [ ]:
from semantic_kernel import __version__

__version__

당신의 Semantic Kernel 버전은 최소 1.27.2여야 합니다.


.env 파일 설정과 리소스를 로드하세요. 키와 설정을 추가하고 로컬 .env 파일을 생성했는지 확인하세요.


In [ ]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Azure에 로그인하기

이제 Azure에 로그인해야 합니다. 터미널을 열고 다음 명령어를 실행하세요:

```bash
az login
```

이 명령어는 Azure 자격 증명을 입력하라는 메시지를 표시하며, 이를 통해 Azure AI Agent 서비스가 올바르게 작동할 수 있습니다.


# 설명:
이 변수는 SERP(검색 엔진 결과 페이지) API 서비스를 이용하기 위한 API 키를 저장합니다. API 키는 계정과 연관된 요청을 인증하는 데 사용되는 고유 식별자입니다.

목적: 이 줄의 목적은 API 키를 변수에 저장하여 SERP API 서비스에 요청을 인증하는 데 사용하는 것입니다. API 키는 서비스를 이용하고 검색을 수행하기 위해 반드시 필요합니다.

SERP API 키를 얻는 방법: SERP API 키를 얻으려면 https://serpapi.com에서 다음 일반적인 단계를 따르세요(사용 중인 특정 SERP API 서비스에 따라 세부 단계는 다를 수 있습니다):

1. SERP API 서비스 선택: SerpAPI, Google Custom Search JSON API 등 여러 SERP API 서비스가 있습니다. 자신의 필요에 가장 적합한 서비스를 선택하세요.

2. 계정 가입: 선택한 SERP API 서비스의 웹사이트로 이동하여 계정을 만드세요. 기본 정보를 제공하고 이메일 주소를 인증해야 할 수도 있습니다.

3. API 키 생성: 가입 후 계정에 로그인하여 API 섹션 또는 대시보드로 이동하세요. 새 API 키를 생성하거나 발급받는 옵션을 찾으세요.

4. API 키를 .env 파일에 복사하세요.


In [ ]:
SERP_API_KEY='SERPAPI_SEARCH_API_KEY'

# 설명:
BASE_URL: 이 변수는 SERP API 엔드포인트의 기본 URL을 저장하는 변수입니다. 변수명 BASE_URL은 이 URL이 API 요청을 시작하는 지점임을 나타내는 관례적인 이름입니다.  
'https://serpapi.com/search':  

이것은 BASE_URL 변수에 할당된 실제 URL 문자열입니다. SERP API를 사용하여 검색 쿼리를 수행하기 위한 엔드포인트를 나타냅니다.

# 목적:
이 줄의 목적은 SERP API의 기본 URL을 저장하는 상수를 정의하는 것입니다. 이 URL은 검색 작업을 수행하기 위해 API 요청을 구성할 때 시작점으로 사용됩니다.

# 사용법:
기본 URL을 변수로 정의하면, SERP API에 요청을 보낼 때마다 코드를 통해 쉽게 재사용할 수 있습니다. 이렇게 하면 코드의 유지보수가 더 쉬워지고, 여러 곳에 URL을 하드코딩함으로써 발생할 수 있는 오류의 위험이 줄어듭니다. 현재 예시는 Bing 검색 API를 사용하는 https://serpapi.com/search?engine=bing 입니다. 다른 API는 https://Serpapi.com 에서 선택할 수 있습니다.


In [ ]:
BASE_URL = 'https://serpapi.com/search?engine=bing'

# 설명:

이곳은 플러그인 코드가 위치한 곳입니다.

클래스 정의: `class BookingPlugin`: 호텔과 항공편 예약 메서드를 포함하는 BookingPlugin이라는 클래스를 정의합니다.

호텔 예약 메서드:

- `@kernel_function(description="booking hotel")`: 호텔 예약을 위한 커널 함수로 기능을 설명하는 데코레이터입니다.
- `def booking_hotel(self, query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-out Time"]) -> Annotated[str, "Return the result of booking hotel information"]:`: 주석이 달린 매개변수와 반환 타입을 사용하여 호텔 예약 메서드를 정의합니다.

이 메서드는 호텔 예약 요청을 위한 매개변수 딕셔너리를 구성하고 SERP API에 GET 요청을 보냅니다. 응답 상태를 확인하고 성공 시 호텔 속성을 반환하며, 요청이 실패하면 None을 반환합니다.

항공편 예약 메서드:

- `@kernel_function(description="booking flight")`: 항공편 예약을 위한 커널 함수로 기능을 설명하는 데코레이터입니다.
- `def booking_flight(self, origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"]) -> Annotated[str, "Return the result of booking flight information"]:`: 주석이 달린 매개변수와 반환 타입을 사용하여 항공편 예약 메서드를 정의합니다.

이 메서드는 출발 및 귀환 항공편 요청을 위한 매개변수 딕셔너리를 구성하고 SERP API에 GET 요청을 보냅니다. 응답 상태를 확인하고 성공 시 항공편 정보를 결과 문자열에 추가하며, 요청이 실패하면 오류 메시지를 출력합니다. 이 메서드는 항공편 정보를 포함하는 결과 문자열을 반환합니다.


In [ ]:
import requests

from typing import Annotated

from semantic_kernel.functions import kernel_function

# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""

    @kernel_function(description="booking hotel")
    def booking_hotel(
        self, 
        query: Annotated[str, "The name of the city"], 
        check_in_date: Annotated[str, "Hotel Check-in Time"], 
        check_out_date: Annotated[str, "Hotel Check-out Time"],
    ) -> Annotated[str, "Return the result of booking hotel information"]:
        """
        Function to book a hotel.
        Parameters:
        - query: The name of the city
        - check_in_date: Hotel Check-in Time
        - check_out_date: Hotel Check-out Time
        Returns:
        - The result of booking hotel information
        """

        # Define the parameters for the hotel booking request
        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "1",
            "currency": "GBP",
            "gl": "uk",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request to the SERP API
        response = requests.get(BASE_URL, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the response content as JSON
            response = response.json()
            # Return the properties from the response
            return response["properties"]
        else:
            # Return None if the request failed
            return None

    @kernel_function(description="booking flight")
    def booking_flight(
        self, 
        origin: Annotated[str, "The name of Departure"], 
        destination: Annotated[str, "The name of Destination"], 
        outbound_date: Annotated[str, "The date of outbound"], 
        return_date: Annotated[str, "The date of Return_date"],
    ) -> Annotated[str, "Return the result of booking flight information"]:
        """
        Function to book a flight.
        Parameters:
        - origin: The name of Departure
        - destination: The name of Destination
        - outbound_date: The date of outbound
        - return_date: The date of Return_date
        - airline: The preferred airline carrier
        - hotel_brand: The preferred hotel brand
        Returns:
        - The result of booking flight information
        """
        
        # Define the parameters for the outbound flight request
        go_params = {
            "engine": "google_flights",
            "departure_id": "destination",
            "arrival_id": "origin",
            "outbound_date": "outbound_date",
            "return_date": "return_date",
            "currency": "GBP",
            "hl": "en",
            "airline": "airline",
            "hotel_brand": "hotel_brand",
            "api_key": "SERP_API_KEY"
        }
 
        print(go_params)

        # Send the GET request for the outbound flight
        go_response = requests.get(BASE_URL, params=go_params)

        # Initialize the result string
        result = ''

        # Check if the outbound flight request was successful
        if go_response.status_code == 200:
            # Parse the response content as JSON
            response = go_response.json()
            # Append the outbound flight information to the result
            result += "# outbound \n " + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Define the parameters for the return flight request
        back_params = {
            #"engine": "google_flights",
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": outbound_date,
            "return_date": return_date,
            "currency": "GBP",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request for the return flight
        back_response = requests.get(BASE_URL, params=back_params)

        # Check if the return flight request was successful
        if back_response.status_code == 200:
            # Parse the response content as JSON
            response = back_response.json()
            # Append the return flight information to the result
            result += "\n # return \n" + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Print the result
        print(result)

        # Return the result
        return result


# 설명:
Import Statements: Azure 자격 증명, AI 에이전트, 채팅 메시지 내용, 작성자 역할, 커널 함수 데코레이터에 필요한 모듈을 가져옵니다.

비동기 컨텍스트 관리자:  
`async with (DefaultAzureCredential() as creds, AzureAIAgent.create_client(credential=creds, conn_str="...") as client,)`:  
Azure 자격 증명을 처리하고 AI 에이전트 클라이언트를 생성하기 위한 비동기 컨텍스트 관리자를 설정합니다.

에이전트 이름 및 지침:  
- `AGENT_NAME = "BookingAgent"`: 에이전트의 이름을 정의합니다.  
- `AGENT_INSTRUCTIONS = """..."""`: 예약 요청을 처리하는 방법에 대한 에이전트의 자세한 지침을 제공합니다.

에이전트 정의 생성:  
`agent_definition = await client.agents.create_agent(...)`: 지정된 모델, 이름, 지침으로 에이전트 정의를 생성합니다.

AzureAI 에이전트 생성:  
`agent = AzureAIAgent(...)`: 클라이언트, 에이전트 정의, 정의된 플러그인을 사용하여 AzureAI 에이전트를 생성합니다.

스레드 생성:  
`thread: AzureAIAgentThread | None = None`: 에이전트를 위한 스레드를 생성합니다. 스레드를 먼저 생성할 필요는 없으며, 값이 `None`으로 제공되면 첫 번째 호출 시 새 스레드가 생성되어 응답의 일부로 반환됩니다.

사용자 입력:  
`user_inputs = ["..."]`: 에이전트가 처리할 사용자 입력 목록을 정의합니다.

마지막으로, 리소스를 정리하기 위해 `finally` 블록에서 스레드와 에이전트를 삭제합니다.


# 인증

`DefaultAzureCredential` 클래스는 Azure SDK for Python의 일부입니다. 이 클래스는 Azure 서비스에 인증하는 기본 방법을 제공합니다. 환경 변수, 관리 ID, Azure CLI 자격 증명 등 특정 순서로 여러 방법을 사용하여 인증을 시도합니다.

비동기 작업: aio 모듈은 DefaultAzureCredential 클래스가 비동기 작업을 지원한다는 것을 나타냅니다. 이를 통해 asyncio를 사용하여 비차단 인증 요청을 수행할 수 있습니다.


In [ ]:
# Import necessary modules
from azure.identity.aio import DefaultAzureCredential
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings, AzureAIAgentThread

ai_agent_settings = AzureAIAgentSettings.create()

# Azure AI Setting
async with (
     DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):    
    
    # Define the agent's name and instructions
    AGENT_NAME = "BookingAgent"
    AGENT_INSTRUCTIONS = """
    You are a booking agent, help me to book flights or hotels.

    Thought: Understand the user's intention and confirm whether to use the reservation system to complete the task.

    Action:
    - If booking a flight, convert the departure name and destination name into airport codes.
    - If booking a hotel or flight, use the corresponding API to call. Ensure that the necessary parameters are available. If any parameters are missing, use default values or assumptions to proceed.
    - If it is not a hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
    - For flight bookings, separate the outbound and return contents and list them in the order of Departure_airport Name | Airline | Flight Number | Departure Time | Arrival_airport Name | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
    - For hotel bookings, list them in the order of Properties Name | Properties description | check_in_time | check_out_time | prices | nearby_places | hotel_class | gps_coordinates.
    """

    # Create agent definition with the specified model, name, and instructions
    agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=AGENT_NAME,
        instructions=AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent using the client and agent definition
    agent = AzureAIAgent(
        client=client,
        definition=agent_definition,
        plugins=[BookingPlugin()]
    )

    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: AzureAIAgentThread | None = None

    # This is your prompt for the activity or task you want to complete 
    # Define user inputs for the agent to process we have provided some example prompts to test and validate 
    user_inputs = [
        # "Can you tell me the round-trip air ticket from  London to New York JFK aiport, the departure time is February 17, 2025, and the return time is February 23, 2025"
        # "Book a hotel in New York from Feb 20,2025 to Feb 24,2025"
        "Help me book flight tickets and hotel for the following trip London Heathrow LHR Feb 20th 2025 to New York JFK returning Feb 27th 2025 flying economy with British Airways only. I want a stay in a Hilton hotel in New York please provide costs for the flight and hotel"
        # "I have a business trip from London LHR to New York JFK on Feb 20th 2025 to Feb 27th 2025, can you help me to book a hotel and flight tickets"
    ]

    try:
        # Process each user input
        for user_input in user_inputs:
            print(f"# User: '{user_input}'")
            # Get the agent's response for the specified thread
            response = await agent.get_response(
                messages=user_input,
                thread=thread,
            )
            thread = response.thread
            # Print the agent's response
            print(f"{response.name}: '{response.content}'")
    finally:
        # Clean up by deleting the thread and agent
        await thread.delete() if thread else None
        await client.agents.delete_agent(agent.id)


---

**면책 조항**:  
이 문서는 AI 번역 서비스 [Co-op Translator](https://github.com/Azure/co-op-translator)를 사용하여 번역되었습니다. 정확성을 위해 최선을 다하고 있으나, 자동 번역에는 오류나 부정확성이 포함될 수 있습니다. 원본 문서의 원어 버전을 신뢰할 수 있는 권위 있는 자료로 간주해야 합니다. 중요한 정보의 경우, 전문적인 인간 번역을 권장합니다. 이 번역 사용으로 인해 발생하는 오해나 잘못된 해석에 대해 책임을 지지 않습니다.
